# Regression, classification


## This exercise uses Berkeley data science teaching materials

This exercise contains parts of labs 11 and 12 from section 3 of
<https://github.com/data-8/materials-x19> with thanks.  See that URL for more
on the license for these materials.  We release this version, like the
original, under the  [Creative Commons Attribution-NonCommercial 4.0
International license](https://creativecommons.org/licenses/by-nc/4.0) (CC
BY-NC 4.0).

# Welcome

Please complete this notebook by filling in the cells provided. Before you
begin, execute the following cell to load the provided tests. Each time you
start your server, you will need to execute this cell again to load the tests.

Directly sharing answers is not okay, but discussing problems with the course
staff or with other students is encouraged. Refer to the Working Together page
to learn more about how to learn cooperatively.

For all problems that you must write our explanations and sentences for, you **must** provide your answer in the designated space. Moreover, throughout this homework and all future ones, please be sure to not re-assign variables throughout the notebook! For example, if you use `max_temperature` in your answer to one question, do not reassign it later on.

As usual, the tests check if you are in the ballpark, but do not check if you have the correct answer.

If your tests here fail, you will not get marks for the question, but if the
tests pass, you may still not get marks, because your answer may still be
incorrect.

In [ ]:
# Don't change this cell; just run it. 
import numpy as np
import pandas as pd
# Safe settings for Pandas.
pd.set_option('mode.chained_assignment', 'raise')

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Minimize function from Scipy
from scipy.optimize import minimize

# Interactive widgets
#from ipywidgets import interact, interactive, fixed
#import ipywidgets as widgets
# ipywidgets doesn't work in deepnote - AO
# if you are working on your local machine and install ipywidgets you could uncomment this

# Load the OKpy test library and tests.
from client.api.notebook import Notebook
ok = Notebook('regress_class.ok')

## 1. Residuals and linear regression


### Regression Model Diagnostics

Linear regression isn't always the best way to describe the relationship
between two variables. We'd like to develop techniques that will help us
decide whether or not to use a linear model to predict one variable based on
another.

We will use the insight that if a regression fits a set of points well, then
the residuals from that regression line will show no pattern when plotted
against the predictor variable. 

The data frame below contains information about crime rates and median home
values in suburbs of Boston, as of 1970. We will attempt to use linear
regression to predict median home value in terms of crime rate.

See <https://github.com/odsti/datasets/tree/master/boston_housing> for more
details on the dataset.

#### About the dataset

We will start by looking at two columns in this dataset - `crime_rate` and `median_home_value`.

Crime rates are per capita per year; home values are in thousands of dollars.
The crime data come from the FBI, and home values are from the US Census
Bureau.

Run the next cell to load the data.

In [ ]:
boston = pd.read_csv('boston_corrected.csv')
boston.head()

Here is a scatterplot of the two columns of interest to us at the moment:

In [ ]:
boston.plot.scatter('crime_rate', 'median_home_value');

#### Question 0

Remember, the correlation is a measure of linear association between two sequences.  Here our sequences are the values for the `crime_rate` column and the values for the `median_home_value` column.

Also remember, that the correlation can be deceiving in various ways.  One way
it can be deceiving is if the relationship between the two sequences is not
linear (a straight line), but some other shape.

Calculate the correlation between `crime_rate` and `median_home_value`, using
any method you like.

*Hint*: You might want to use the routines from previous notebooks in the
textbook.

In [ ]:
r_cr_mhv = ...
# Show the result.
r_cr_mhv

In [ ]:
_ = ok.grade('q1_0')

#### Question 1

The *residuals* are the values that *remain* in a sequence of values, after we
have removed whatever we can predict using other values.

They have two uses that you will see in this notebook:

* We can use the residuals to *look for patterns*.   These patterns can
  suggest problems in our original predictions.
* We can use residuals to investigate what associations *remain* after
  removing another association.

In this section we will use residuals as a tool to look for patterns.

Write a function called `calc_residuals`.  It should take three arguments:

* `input_df`: a data frame
* `x_col_name`: a string giving the name of column with the values we will use
  to predict with.
* `y_col_name`: a string giving the name of column with the values we will use
  to predict.

`calc_residuals` should first compute the slope and intercept of the least
squares regression line that predicts the `y_col_name` column of the data frame
using the `x_col_name` column of the data frame. The function should return an
array containing the *residuals* for that regression line. Recall that
residuals are given by 

$$residual = observed \ value - regression \ estimate$$

We have also called the "regression estimate" above - the "predicted" values.

*Hint*: For your convenience, in case you want to use it, here is the
standard `ss_any_line` function we have used before:

In [ ]:
def ss_any_line(c_s, x_values, y_values):
    c, s = c_s
    predicted = c + x_values * s
    error = y_values - predicted
    return np.sum(error ** 2)

In [ ]:
def calc_residuals(input_df, x_col_name, y_col_name):
    ...

In [ ]:
_ = ok.grade('q1_1')

#### Question 2

Make a scatter plot of the residuals from predicting `median_home_value` from
the `crime_rate`.  Plot the residuals on the vertical axis, and the crime rate
on the horizontal axis.

In [ ]:
...

#### Question 3

Does the plot of residuals look roughly like a formless cloud, or is there
some kind of pattern? Are they centered around 0?

*Write your answer here, replacing this text.*

#### Question 4

Does it seem like a linear model is appropriate for describing the
relationship between crime and median home value?

Assign `linear` to `True` if a linear model is appropriate for describing the
relationship, and `False` if it is not.

In [ ]:
linear = ...

In [ ]:
_ = ok.grade('q1_4')

#### Question 5

So far you have used residuals to look for patterns in the remaining data.  This can give us clues as to where our original predictions are failing.

We can also use residuals as a measure for what remains, after removing a particular effect.  That is the use we will study here.

You have just looked at the crime rate as a predictor of house prices, but
there are many other columns in this dataset with measures that we would also
expect to be relatable to house prices.

Perhaps these other factors can explain a large proportion of the variation in home prices.  Perhaps, if we remove the effects of these other factors, this will reveal a stronger association between crime rate and home prices.  Or perhaps, these other factors are also related to the crime rate, and when we remove these other factors, the association of home prices with crime rate will decrease, because we have already removed the common source of variation.

One factor we would expect to predict home prices is the average size of the houses, as measured by the average number of rooms.  This measure is in the `n_rooms_avg` column of the dataset.   If the "town" tends to have homes with more rooms, we expect these homes to be more expensive on average.

First investigate whether `n_rooms_avg` is a possible explanatory factor in
`median_home_prices`.  Calculate the correlation between `n_rooms_avg` and
`median_home_value`, using any method you like.  Store the correlation
coefficient as `r_nra_mhv` (the name is an abbreviation of "r of `n_rooms_avg`
with `median_home_value`").

In [ ]:
r_nra_mhv = ...
# Show the result.
r_nra_mhv

In [ ]:
_ = ok.grade('q1_5')

#### Question 6

Your correlation might have persuaded you that we can predict house prices, to
some degree, when we know the `n_rooms_avg` value.

Now we wonder - does the apparent relationship between `crime_rate` and
`median_home_value` still hold, when we allow for the values in
`n_rooms_avg`?

Address this question by *removing* the effect of `n_rooms_avg` on
`median_home_value`.  Do this by taking the residuals of the
`median_home_values` after predicting from `n_rooms_avg`.  Then we look at the
correlation of the *residuals* with `crime_rate`.  Calculate and store this
correlation as `r_cr_r_nra` (an abbreviation of "r of `crime_rate` with the residuals of regressing `n_rooms_avg` on `median_home_value`").

In [ ]:
#- Calculate the residuals for median_home_value after predicting from
#- n_rooms_avg.  Calculate the correlation of these residuals with
#- crime_rate.
r_cr_r_nra = ...
# Show the result
r_cr_r_nra

In [ ]:
_ = ok.grade('q1_6')

#### Question 7

You may have seen above, that once we have adjusted home values with what we
can predict from `n_rooms_avg`, the association of the remaining home prices
with `crime_rate` is a little less strong, but similar.

There are other factors recorded in the dataset that might explain house
prices.  Some of these could have a stronger relationship with `crime_rate`,
and therefore, explain more of the relationship between `crime_rate` and
`median_home_value`.

For example, one rather eye-watering column is `lower_stat_pct`.  It is an
attempt to measure the percentage of homes in a given Boston suburb owned by
people of "lower social status".  Specifically, it measures the percentage of
home owners that do not have a high-school education, or that have work
classified as "laborers".  We might expect that people in these categories
would have less money, and therefore, will tend to live in suburbs with
cheaper houses.

We could also imagine that people with lower incomes will have greater need of
money and therefore, more incentive to the kind of crime that affects
`crime_rate`.   Maybe, if we remove the effect of `lower_stat_pct` from house
prices, we will thereby remove the effect of `crime_rate`.

Calculate the residuals after regressing `lower_stat_pct` against `median_home_value`, and then calculate the correlation of these residuals with `crime_rate`.

In [ ]:
#- Calculate the residuals for median_home_value after predicting from
#- lower_stat_pct.  Calculate the correlation of these residuals with
#- crime_rate.
r_cr_r_lsp = ...
# Show the result
r_cr_r_lsp

In [ ]:
_ = ok.grade('q1_7')

#### Question 8

We hope you found that adjusting for `lower_stat_pct` had a more dramatic effect on the correlation with `crime_rate`.

However, these effects of adjusting for different variables can be complicated.

So far you have seen the correlations can go down when adjusting for other effects that have some relationship with the predictor we are most interested in - `crime_rate`.

In fact, the relationship between variables can be rather complicated - and correlations can go up or down after adjusting for other variables.

For example, another step we might take is to use *multiple regression* to
adjust `median_home_value` *both* for `n_rooms_avg` *and* `lower_stat_pct`.
After we have done that, we can look at the residuals from the multiple
regression, and see whether these residuals, adjusted for both effects,
correlate with `crime_rate`.

[Remember](https://uob-ds.github.io/cfd2021/classification/single_multiple.html) that, with
multiple regression, we use more than one predictor column to predict the
values we are interested in.  In our case we are going to use the
`n_rooms_avg` and the `lower_stat_pct` predictor columns to predict the values
in the `median_home_value` column.

We need to find three values to do this prediction:

* A "best" intercept. Call this `intercept`
* A "best" slope for `n_rooms_avg`.  Call this `slope_room`.
* A "best" slope for `lower_stat_pct`.  Call this `slope_stat`.

We will define "best", as usual, as the `intercept`, `slope_room` and
`slope_stat` such that our predictions for `median_home_value` using these
values, minimize the sums of squared differences from the actual `median_home_value` values.

For any given `intercept`, `slope_room` and `slope_stat`, our prediction for `median_home_value` will be:

```
intercept + slope_room * boston['n_rooms_avg'] + slope_stat * boston['lower_stat_pct']
```

Call the "best" three values `[intercept, slope_room, slope_stat]` - the
*parameters* we are trying to find.

We will use `minimize` to find our "best" parameters.

First we need a function `ss_ls_rm_mhv` that takes an array `params`, with
three values, and calculates the sum of squared errors.   For simplicity, the
function should take a single parameter `params` that is an array with 3
elements.   The three elements are respectively:

* The value for the intercept
* The value for the slope for `n_rooms_avg`
* The value for the slope for `lower_stat_pct`

The function will use the `boston` data frame from the top level to get the
relevant columns in order to do the calculation.   It returns a single number,
the sum of squared differences between the prediction, given the `params`, and
the `boston['median_home_value']` column values.


In [ ]:
def ss_ls_rm_mhv(params):
    ...

In [ ]:
# Evaluate your function for the parameters [1, 0.5, -0.5]
# (intercept of 1, slope_room of 0.5, slope_stat of -0.5)
ss_ls_rm_mhv([1, 0.5, -0.5])

In [ ]:
_ = ok.grade('q1_8')

#### Question 9

Now you have a function you can use with `minimize`.

Call `minimize` and store the result as `min_result`.  The result from a call to minimize has various attributes, including `fun` (the lowest value that the function can return) and `x` (the parameters that result in this lowest value).

In [ ]:
min_result = ...
# Show the returned value.
min_result

In [ ]:
_ = ok.grade('q1_9a')

With the parameters from `min_result`, calculate the residuals for
`median_home_value`. Remember, the residuals are `median_home_value` values
after subtracting the *prediction* from the intercept and the two slopes.

In [ ]:
multi_residuals = ...
# Show the first five residuals
multi_residuals[:5]

In [ ]:
_ = ok.grade('q1_9b')

Remember the `fun` attribute of `min_result`.  This is the lowest value that `minimize` could get when calling the function you gave it.  `x` is the array of parameters to pass to the function to give this `fun` value.

You might want to confirm to yourself that the sum of squares of the residuals you have just calculated for the "best" parameters are indeed very close to the value for the `fun` attribute of `min_result`:

In [ ]:
print('Lowest value from min_result', min_result.fun)
print('Sum of squares of residuals', np.sum(multi_residuals ** 2))

Investigate the effect of adjusting for both `n_rooms_avg` and
`lower_stat_pct` by calculating the correlation coefficient between the
residuals from the multiple regression and `crime_rate`.

In [ ]:
r_cr_r_nra_lsp = ...
# Show the result
r_cr_r_nra_lsp

In [ ]:
_ = ok.grade('q1_9c')

**For reflection**: Has the correlation gone up or down when we added the
adjustment for both columns (`n_rooms_avg` and `lower_stat_pct`).  Think about
why that might be.  *Warning* - these multiple regression relationships can be
hard to think about.


## 2. Reading Sign Language with Classification

Brazilian Sign Language is a visual language used primarily by Brazilians who
are deaf.  It is more commonly called Libras.  People who communicate with
visual language are called *signers*.  Here is a video of someone signing in
Libras: https://www.youtube.com/watch?v=SiJUggsJ3e8

Programs like Siri or Google Now begin the process of understanding human
speech by classifying short clips of raw sound into basic categories called
*phones*.  For example, the recorded sound of someone saying the word "robot"
might be broken down into several phones: "rrr", "oh", "buh", "aah", and
"tuh".  Phones are then grouped together into further categories like words
("robot") and sentences ("I, for one, welcome our new robot overlords") that
carry more meaning.

A visual language like Libras has an analogous structure.  Instead of phones,
each word is made up of several *hand movements*.  As a first step in
interpreting Libras, we can break down a video clip into small segments, each
containing a single hand movement.  The task is then to figure out what hand
movement each segment represents.

We can do that with classification!

The [data](https://archive.ics.uci.edu/ml/machine-learning-databases/libras/movement_libras.names) in this exercise come from Dias, Peres, and Biscaro, researchers at the University of Sao Paulo in Brazil.  They identified 15 distinct hand movements in Libras (probably an oversimplification, but a useful one) and captured short videos of signers making those hand movements.  (You can read more about their work [here](http://ieeexplore.ieee.org/Xplore/login.jsp?url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F5161636%2F5178557%2F05178917.pdf&authDecision=-203). The paper is gated, so you will need to use your institution's Wi-Fi or VPN to access it.)

For each video, they chose 45 still frames from the video and identified the
location (in horizontal and vertical coordinates) of the signer's hand in each
frame.  Since there are two coordinates for each frame, this gives us a total
of 90 numbers summarizing how a hand moved in each video.  Those 90 numbers
will be our *attributes*.

Each video is *labeled* with the kind of hand movement the signer was making in it.  Each label is one of 15 strings like "horizontal swing" or "vertical zigzag".

For simplicity, we're going to focus on distinguishing between just two kinds
of movements: "horizontal straight-line" and "vertical straight-line".  We
took the Sao Paulo researchers' original dataset, which was quite small, and
used some simple techniques to create a much larger synthetic dataset.

These data are in the file `movements.csv`.  Run the next cell to load it.

In [ ]:
movements = pd.read_csv("movements.csv")
movements.head()

The cell below displays movements graphically.  Run it and use the slider to
answer the next question.

In [ ]:
# Change the X value of display_whole_movement(X) to see plots of different movements
# e.g. display_whole_movement(2) to see the third movement
# Run this cell a number of times, changing the movement number, to get a feeling for what
# different movements look like

def display_whole_movement(row_idx):
    n, p = movements.shape
    num_frames = int((p - 1)/2)
    row = movements.drop("Movement type", axis=1).iloc[row_idx]
    xs = row.iloc[np.arange(0, 2*num_frames, 2)]
    ys = row.iloc[np.arange(1, 2*num_frames, 2)]
    plt.figure(figsize=(5,5))
    plt.plot(xs, ys, c="gold")
    xmax=np.max(xs)
    xmin=np.min(xs)
    ymax=np.max(ys)
    ymin=np.min(ys)
    xrange=xmax-xmin
    yrange=ymax-ymin
    plot_margin = 0.2
    if(xrange > yrange):
        half_range=(xrange*(1+plot_margin))/2
    else:
        half_range=(yrange*(1+plot_margin))/2
    xmidpoint=xmin+xrange/2
    ymidpoint=ymin+yrange/2
    plt.xlabel("x")
    plt.ylabel("y")
    xplotmin=xmidpoint-half_range
    xplotmax=xmidpoint+half_range
    yplotmin=ymidpoint-half_range
    yplotmax=ymidpoint+half_range
    plt.xlim(xplotmin, xplotmax)
#    plt.ylim(-.5, 1.5)
    plt.ylim(yplotmin,yplotmax)
    plt.gca().set_aspect('equal', adjustable='box')
    
display_whole_movement(6)

#### Question 1

Before we move on, check your understanding of the dataset.  Judging by the
plot, is the first movement example a vertical motion, or a horizontal motion?
If it is hard to tell, does it seem more likely to be vertical or horizontal?
This is the kind of question a classifier has to answer.  Find out the right
answer by looking at the `"Movement type"` column.

Assign `first_movement` to `1` if the movement was vertical, or `2` if the movement was horizontal.

In [ ]:
first_movement = ...

In [ ]:
_ = ok.grade('q2_1')

### Splitting the dataset

We'll do 2 different kinds of things with the `movements` dataset:

1. We'll build a classifier that uses the movements with known labels as
   examples to classify similar movements.  This is called *training*.
2. We'll evaluate or *test* the accuracy of the classifier we build.

For reasons discussed in lecture and the textbook, we want to use separate
datasets for these two purposes.  So we split up our one dataset into two.


#### Question 2

Create a data frame called `train_movements` and another data frame called
`test_movements`.  `train_movements` should include the first
$\frac{11}{16}$th of the rows in `movements` (rounded to the nearest integer),
and `test_movements` should include the remaining $\frac{5}{16}$th.

Note that we do **not** mean the first 11 rows for the training test and rows
12-16 for the test set. We mean the first $\frac{11}{16} = 68.75$% of the
data frame should be for the training set, and the rest should be for the test
set.

*Hint:* Use the dataset `iloc` indexing.

In [ ]:
training_proportion = 11/16
num_movements = len(movements)
num_train = int(round(num_movements * training_proportion))
train_movements = ...
test_movements = ...
print("Training set:\t", len(train_movements), "examples")
print("Test set:\t", len(test_movements), "examples")

In [ ]:
_ = ok.grade('q2_2')

### Using only 2 features

First let's see how well we can distinguish two movements (a vertical line and a horizontal line) using the hand position from just a single frame (without the other 44).


#### Question 3

Make a data frame called `train_two_features` with only 3 columns: the first
frame’s x coordinate and first frame’s y coordinate are our chosen features,
as well as the movement type; only the examples in `train_movements`.

You might want to scroll up to check the column names in the data frame.

In [ ]:
train_two_features = ...
train_two_features

In [ ]:
_ = ok.grade('q2_3')

Now we want to make a scatter plot of the frame coordinates, where the dots
for horizontal straight-line movements have one color and the dots for
vertical straight-line movements have another color.  Here is a scatter plot
without colors:

In [ ]:
train_two_features.plot.scatter("Frame 1 x", "Frame 1 y")

This isn't useful because we don't know which dots are which movement type.
We need to tell Python how to color the dots.  Let's use gold for vertical and
blue for horizontal movements.

`scatter` takes an extra argument called `c` that's the name of an extra
column in the data frame that contains colors (strings like "red" or "orange")
for each row.  So we need to create a data frame like this:

|Frame 1 x|Frame 1 y|Movement type|Color|
|-|-|-|-|
|0.522768|0.769731|vertical straight-line|gold|
|0.179546|0.658986|horizontal straight-line|blue|
|...|...|...|...|


#### Question 4

In the cell below, create a *new* data frame named `with_colors`.  It should
have the same columns as the example data frame above, but with a row for each
row in `train_two_features`. Then, create a scatter plot of your data.

In [ ]:
with_colors = train_two_features.copy()
with_colors = ...

#### Question 5

Based on the scatter plot, how well will a nearest-neighbor classifier based on only these 2 features (the x- and y-coordinates of the hand position in the first frame) work?  Will it:

1. distinguish almost perfectly between vertical and horizontal movements;
2. distinguish somewhat well between vertical and horizontal movements,
   getting some correct but missing a substantial proportion; or
3. be basically useless in distinguishing between vertical and horizontal
   movements?

Why?

*Write your answer here, replacing this text.*

## 3. Classification Potpourri

Throughout this question, we will aim to discuss some conceptual nuances of
classification that often get overlooked when we're focused only on improving
our accuracy and building the best classifier possible. 


#### Question 1

What is the point of a test-set? Should we use our test set to find the best
possible number of neighbors for a k-NN classifier? Explain.

*Write your answer here, replacing this text.*

#### Question 2

You have a large dataset `breast-cancer` which has three columns. The first
two are attributes of the person that might be predictive of whether or not
someone has breast-cancer, and the third column indicates whether they have it
or not. 99% of the data frame contains examples of people who do not have
breast cancer.

Imagine you are trying to use a k-NN classifier to use the first two columns
to predict whether or not someone has breast cancer. You split your training
and test set up as necessary, you develop a 7 Nearest Neighbors classifier,
and you notice your classifier predicts every point in the test set to be a
person who does not have breast cancer. Is there a problem with your
classifier? Explain this phenomenon.

*Write your answer here, replacing this text.*

#### Question 3

You have a training set of 35 examples of characteristics of fruits along with
what fruit is actually being described. 25 of the examples of Apples, and 10
of the examples are Oranges.

You decide to make a k-NN classifier. Assign `k_upper_bound` to the smallest
value of k such that the classifier will *always* predict Apple for *every*
point, regardless of how the data is actually spread out.

Imagine that ties are broken at random for even values of k, so there is no
guarantee of what will be picked if there is a tie.


In [ ]:
k_upper_bound = ...

In [ ]:
_ = ok.grade('q3_3')

## Done

You're finished with the assignment!  Be sure to...

- **run all the tests** (the next cell has a shortcut for that),
- **Save and Checkpoint** from the "File" menu.
- Finally, **restart** the kernel for this notebook, and **run all the
  cells**, to check that the notebook still works without errors.  Use the
  "Kernel" menu, and choose "Restart and run all".  If you find any problems,
  go back and fix them, save the notebook, and restart / run all again, before
  submitting.  When you do this, you make sure that we, your humble markers,
  will be able to mark your notebook.

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1c17e9cc-849f-47ac-b8e8-1c59753809f4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>